# Emoji Prediction

In [1]:
# !pip install emoji

In [2]:
import emoji
import numpy as np
import pandas as pd

In [3]:
mapping = pd.read_csv("../input/twitter-emoji-prediction/Mapping.csv")
output = pd.read_csv("../input/twitter-emoji-prediction/OutputFormat.csv")
train = pd.read_csv("../input/twitter-emoji-prediction/Train.csv")
test = pd.read_csv("../input/twitter-emoji-prediction/Test.csv")

In [4]:
mapping

Unnamed: 0 emoticons  number
0            0         😜       0
1            1         📸       1
2            2         😍       2
3            3         😂       3
4            4         😉       4
5            5         🎄       5
6            6         📷       6
7            7         🔥       7
8            8         😘       8
9            9         ❤       9
10          10         😁      10
11          11        🇺🇸      11
12          12         ☀      12
13          13         ✨      13
14          14         💙      14
15          15         💕      15
16          16         😎      16
17          17         😊      17
18          18         💜      18
19          19         💯      19

In [5]:
mapping = mapping.drop("Unnamed: 0", axis=1)
mapping.head()

emoticons  number
0         😜       0
1         📸       1
2         😍       2
3         😂       3
4         😉       4

In [6]:
mapping = mapping.set_index('number')

In [7]:
mapping.head()

emoticons
number          
0              😜
1              📸
2              😍
3              😂
4              😉

In [8]:
mapping_dict = mapping.to_dict()
mapping_dict = mapping_dict['emoticons']
mapping_dict

{0: '😜',
 1: '📸',
 2: '😍',
 3: '😂',
 4: '😉',
 5: '🎄',
 6: '📷',
 7: '🔥',
 8: '😘',
 9: '❤',
 10: '😁',
 11: '🇺🇸',
 12: '☀',
 13: '✨',
 14: '💙',
 15: '💕',
 16: '😎',
 17: '😊',
 18: '💜',
 19: '💯'}

In [9]:
train.head()

Unnamed: 0                                               TEXT  Label
0           0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1           1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2           2  Been friends since 7th grade. Look at us now w...      2
3           3  This is what it looks like when someone loves ...      3
4           4  RT @user this white family was invited to a Bl...      3

In [10]:
train.columns

Index(['Unnamed: 0', 'TEXT', 'Label'], dtype='object')

In [11]:
train = train.drop('Unnamed: 0', axis=1)

In [12]:
train.tail()

TEXT  Label
69995  Yes, I call Galina "my Bubie" Go follow my bea...      3
69996    I SEA you, Seattle @ Ballard Seafood Festival\n     16
69997  If one of my daughters is wearing this and ask...      2
69998  Guess who whoop people on THEIR homecoming?! #...      3
69999  We Love you Robbie @ Heritage Memorial Cemeter...     14

In [13]:
# changing the labels to emoji
# train = train.replace({"Label": mapping_dict})

In [14]:
train.head()

TEXT  Label
0  Vacation wasted ! #vacation2017 #photobomb #ti...      0
1  Oh Wynwood, you’re so funny! : @user #Wynwood ...      1
2  Been friends since 7th grade. Look at us now w...      2
3  This is what it looks like when someone loves ...      3
4  RT @user this white family was invited to a Bl...      3

## Text Preprocessing using Embeddings

In [15]:
X_train = train['TEXT'].values
y_train = train['Label'].values

print(X_train.shape, y_train.shape)

(70000,) (70000,)


In [16]:
# maxLen = X_train.map(len).max()

In [17]:
# maxLen, type(maxLen)

In [18]:
file = open("../input/glove6b50dtxt/glove.6B.50d.txt", encoding = 'utf8')

In [19]:
file

<_io.TextIOWrapper name='../input/glove6b50dtxt/glove.6B.50d.txt' mode='r' encoding='utf8'>

In [20]:
def intialize_emb_matrix(file):
    embedding_matrix = {}
    for line in file:
        values = line.split()
        word = values[0]
        embedding = np.array(values[1:], dtype='float64')
        embedding_matrix[word] = embedding

    return embedding_matrix 


In [21]:
embedding_matrix = intialize_emb_matrix(file)

In [22]:
embedding_matrix['dance']

array([-9.9716e-01,  6.1348e-01, -1.7686e+00,  8.7052e-02, -2.4644e-01,
        1.9984e-01, -2.5892e-01, -3.1309e-02, -4.3117e-01,  1.0844e+00,
        4.7032e-01, -3.6731e-01,  1.0047e-01,  1.2459e+00,  2.5728e-03,
       -3.8805e-01,  4.8489e-01,  3.8052e-01, -5.6983e-01, -8.4332e-01,
        8.8607e-01,  9.7396e-01,  9.0600e-03,  9.5187e-01,  3.3867e-01,
       -2.5218e-01, -1.5633e+00, -4.8921e-01, -3.4712e-01, -1.2629e+00,
        2.8455e+00,  2.8887e-01, -1.9158e-01,  6.5195e-02, -1.9799e-01,
        2.2815e-01,  4.4613e-01, -8.2659e-02, -3.3068e-01,  4.8144e-02,
       -2.7224e-01, -2.1765e-01, -5.5931e-01, -4.0149e-01,  5.6395e-01,
       -1.9915e-01,  7.2645e-01, -1.5184e+00, -8.7863e-01,  2.1860e-01])

In [23]:
embedding_matrix['dance'].shape 

(50,)

In [24]:
def get_emb_data(data, max_len):
#     max_len = 168
    embedding_data = np.zeros((len(data), max_len, 50))  # from glove6B50d
    
    for idx in range(data.shape[0]):
        words_in_sentence = data[idx].split()
        
        for i in range(len(words_in_sentence)):
            if embedding_matrix.get(words_in_sentence[i].lower()) is not None:
                embedding_data[idx][i] = embedding_matrix[words_in_sentence[i].lower()]
                
    return embedding_data

In [25]:
# representing each word in the sentence acc. to glove embedding [str --> numerical]
X_temb = get_emb_data(X_train, 168)

In [26]:
X_temb

array([[[ 1.2875e+00,  1.2586e+00, -7.1297e-01, ..., -2.3279e-01,
          2.2120e-01,  7.2042e-01],
        [ 1.8103e-01, -4.0867e-01,  4.3368e-01, ...,  2.9675e-01,
          1.5272e-01, -1.8731e-01],
        [-5.8402e-01,  3.9031e-01,  6.5282e-01, ..., -1.2338e+00,
          4.6715e-01,  7.8858e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]],

       [[-7.0292e-02,  1.6078e+00,  6.4854e-01, ..., -6.4507e-01,
          4.9646e-01,  7.1087e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  

In [27]:
X_temb.shape

(70000, 168, 50)

In [28]:
# X_temb = X_temb.reshape(-1, 20, 50)
# X_temb.shape

In [29]:
# convert outputs to categorical variables
from keras.utils.np_utils import to_categorical

In [30]:
# converting y_train to one hot vectors so that cross-entropy loss can be used.
y_train = to_categorical(y_train)

In [31]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Model Building

In [32]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense, Dropout

In [33]:
model = Sequential()

2021-11-28 04:25:56.602671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 04:25:56.732415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 04:25:56.733144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 04:25:56.734249: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [34]:
lm = len(mapping_dict)
lm

20

In [35]:
model.add(LSTM(units = 256, return_sequences=True, input_shape = (168,50)))
model.add(Dropout(0.3))
model.add(LSTM(units=128))
model.add(Dropout(0.3))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=20, activation='softmax'))

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 168, 256)          314368    
_________________________________________________________________
dropout (Dropout)            (None, 168, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2

In [37]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [38]:
# model Training
res = model.fit(X_temb, y_train, validation_split=0.2, batch_size=32, epochs=10, verbose=2)

2021-11-28 04:26:01.230542: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1881600000 exceeds 10% of free system memory.
2021-11-28 04:26:03.193466: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1881600000 exceeds 10% of free system memory.
2021-11-28 04:26:04.561731: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2021-11-28 04:26:07.635009: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1750/1750 - 47s - loss: 2.7465 - acc: 0.2147 - val_loss: 2.7387 - val_acc: 0.2183
Epoch 2/10
1750/1750 - 41s - loss: 2.7385 - acc: 0.2149 - val_loss: 2.7369 - val_acc: 0.2183
Epoch 3/10
1750/1750 - 41s - loss: 2.7387 - acc: 0.2149 - val_loss: 2.7381 - val_acc: 0.2183
Epoch 4/10
1750/1750 - 41s - loss: 2.7375 - acc: 0.2149 - val_loss: 2.7367 - val_acc: 0.2183
Epoch 5/10
1750/1750 - 41s - loss: 2.7368 - acc: 0.2149 - val_loss: 2.7362 - val_acc: 0.2183
Epoch 6/10
1750/1750 - 41s - loss: 2.7366 - acc: 0.2149 - val_loss: 2.7355 - val_acc: 0.2183
Epoch 7/10
1750/1750 - 41s - loss: 2.7363 - acc: 0.2149 - val_loss: 2.7370 - val_acc: 0.2183
Epoch 8/10
1750/1750 - 42s - loss: 2.7359 - acc: 0.2149 - val_loss: 2.7361 - val_acc: 0.2183
Epoch 9/10
1750/1750 - 42s - loss: 2.7360 - acc: 0.2149 - val_loss: 2.7367 - val_acc: 0.2183
Epoch 10/10
1750/1750 - 42s - loss: 2.7358 - acc: 0.2149 - val_loss: 2.7360 - val_acc: 0.2183


## Using Another dataset for Text to emoji prediction

In [39]:
train_data = pd.read_csv("../input/emojify/train_emoji.csv", header=None)
test_data = pd.read_csv("../input/emojify/test_emoji.csv", header=None)

In [40]:
train_data.head()

0  1   2     3
0           never talk to me again  3 NaN   NaN
1  I am proud of your achievements  2 NaN   NaN
2   It is the worst day in my life  3 NaN   NaN
3                 Miss you so much  0 NaN   [0]
4                     food is life  4 NaN   NaN

In [41]:
test_data.head()

0  1
0              I want to eat\t  4
1          he did not answer\t  3
2   he got a very nice raise\t  2
3  she got me a nice present\t  2
4   ha ha ha it was so funny\t  2

In [42]:
train_data.drop(labels=[2, 3], axis=1, inplace=True)

In [43]:
train_data.tail()

0  1
127  he had to make a home run  1
128         I am ordering food  4
129     What is wrong with you  3
130                 I love you  0
131                  great job  2

In [44]:
emoji.EMOJI_UNICODE['en']

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰️',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱️',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_

In [45]:
emoji.emojize(':angry_face:')

'😠'

In [46]:
emoji_mapping = {
    '0': ':beating_heart:',
    '1': ':baseball:',
    '2': ':beaming_face_with_smiling_eyes:',
    '3': ':angry_face:',
    '4': ':face_savoring_food:'
}

In [47]:
for key, value in emoji_mapping.items():
    emoji_mapping[key] = emoji.emojize(value)
    
print(emoji_mapping)
    

{'0': '💓', '1': '⚾', '2': '😁', '3': '😠', '4': '😋'}


In [48]:
X_train = train_data[0].values
y_train = train_data[1].values

In [49]:
# def intialize_emb_matrix(file):
#     embedding_matrix = {}
#     for line in file:
#         values = line.split()
#         word = values[0]
#         embedding = np.array(values[1:], dtype='float64')
#         embedding_matrix[word] = embedding

#     return embedding_matrix 

file

<_io.TextIOWrapper name='../input/glove6b50dtxt/glove.6B.50d.txt' mode='r' encoding='utf8'>

In [50]:
# we have already created an embedding matrix using the golve vectors
embedding_matrix

{'the': array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01]),
 ',': array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
        -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
        -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
        -0.41634 , -0.15428 ,  0.1

In [51]:
X_train = get_emb_data(X_train, 10)

In [52]:
X_train

array([[[ 9.5387e-02, -1.6865e-01, -1.1514e-01, ..., -4.2103e-01,
         -5.3817e-01,  1.3738e-01],
        [-3.4604e-02,  4.1087e-01, -9.7368e-02, ...,  5.6204e-01,
         -5.9770e-01,  9.5068e-01],
        [ 6.8047e-01, -3.9263e-02,  3.0186e-01, ..., -7.3297e-02,
         -6.4699e-02, -2.6044e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]],

       [[ 1.1891e-01,  1.5255e-01, -8.2073e-02, ..., -5.7512e-01,
         -2.6671e-01,  9.2121e-01],
        [ 3.4664e-01,  3.9805e-01,  4.8970e-01, ...,  8.6576e-02,
          3.4037e-01,  1.3588e+00],
        [-5.9180e-01,  2.7671e-01, -4.6971e-01, ..., -2.0055e-01,
         -2.6487e-01,  5.7981e-01],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  

In [53]:
y_train = to_categorical(y_train)
y_train

array([[0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0.

In [54]:
model_tte = Sequential()
model_tte.add(LSTM(units=64, input_shape=(10, 50), return_sequences=True))
model_tte.add(Dropout(0.3))
model_tte.add(LSTM(units=32))
model_tte.add(Dropout(0.2))
model_tte.add(Dense(units=10, activation='relu'))
model_tte.add(Dense(units=5, activation='relu'))

In [55]:
model_tte.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 55        
Total params: 42,241
Trainable params: 42,241
Non-trainable params: 0
__________________________________________________

In [56]:
model_tte.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [57]:
history = model_tte.fit(X_train, y_train, validation_split=0.1, verbose=2, batch_size=32, epochs=100)

Epoch 1/100
4/4 - 3s - loss: 7.5985 - acc: 0.2458 - val_loss: 6.6778 - val_acc: 0.0714
Epoch 2/100
4/4 - 0s - loss: 5.2093 - acc: 0.2881 - val_loss: 8.6703 - val_acc: 0.1429
Epoch 3/100
4/4 - 0s - loss: 5.3723 - acc: 0.2797 - val_loss: 6.8144 - val_acc: 0.1429
Epoch 4/100
4/4 - 0s - loss: 4.4341 - acc: 0.3305 - val_loss: 5.5591 - val_acc: 0.0714
Epoch 5/100
4/4 - 0s - loss: 4.2089 - acc: 0.2881 - val_loss: 5.4632 - val_acc: 0.1429
Epoch 6/100
4/4 - 0s - loss: 3.6260 - acc: 0.3305 - val_loss: 5.4198 - val_acc: 0.1429
Epoch 7/100
4/4 - 0s - loss: 3.5119 - acc: 0.3559 - val_loss: 5.4258 - val_acc: 0.1429
Epoch 8/100
4/4 - 0s - loss: 3.5752 - acc: 0.3305 - val_loss: 5.4327 - val_acc: 0.1429
Epoch 9/100
4/4 - 0s - loss: 3.6350 - acc: 0.3475 - val_loss: 5.4195 - val_acc: 0.1429
Epoch 10/100
4/4 - 0s - loss: 3.4894 - acc: 0.3305 - val_loss: 5.4022 - val_acc: 0.2143
Epoch 11/100
4/4 - 0s - loss: 3.1117 - acc: 0.3644 - val_loss: 3.8159 - val_acc: 0.3571
Epoch 12/100
4/4 - 0s - loss: 1.9897 - ac

In [58]:
model_tte2 = Sequential()
model_tte2.add(LSTM(units=128, input_shape=(10, 50), return_sequences=True))
model_tte2.add(Dropout(0.3))
model_tte2.add(LSTM(units=64))
model_tte2.add(Dropout(0.2))
model_tte2.add(Dense(units=32, activation='relu'))
model_tte2.add(Dense(units=20, activation='relu'))
model_tte2.add(Dense(units=5, activation='relu'))

In [59]:
model_tte2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 20)                660       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                

In [60]:
model_tte2.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['acc'])

In [61]:
history2 = model_tte2.fit(X_train, y_train, validation_split=0.1, verbose=2, batch_size=32, epochs=500)

Epoch 1/500
4/4 - 3s - loss: 4.6914 - acc: 0.2797 - val_loss: 5.5716 - val_acc: 0.2143
Epoch 2/500
4/4 - 0s - loss: 2.3278 - acc: 0.2881 - val_loss: 1.8905 - val_acc: 0.1429
Epoch 3/500
4/4 - 0s - loss: 1.5472 - acc: 0.3136 - val_loss: 1.5059 - val_acc: 0.4286
Epoch 4/500
4/4 - 0s - loss: 2.2562 - acc: 0.2288 - val_loss: 1.6962 - val_acc: 0.2143
Epoch 5/500
4/4 - 0s - loss: 1.9568 - acc: 0.2458 - val_loss: 2.5884 - val_acc: 0.1429
Epoch 6/500
4/4 - 0s - loss: 1.9897 - acc: 0.2712 - val_loss: 1.6850 - val_acc: 0.1429
Epoch 7/500
4/4 - 0s - loss: 2.2633 - acc: 0.2797 - val_loss: 1.6773 - val_acc: 0.1429
Epoch 8/500
4/4 - 0s - loss: 1.5810 - acc: 0.2966 - val_loss: 1.6697 - val_acc: 0.2143
Epoch 9/500
4/4 - 0s - loss: 1.5956 - acc: 0.3644 - val_loss: 1.6210 - val_acc: 0.2143
Epoch 10/500
4/4 - 0s - loss: 1.4789 - acc: 0.2966 - val_loss: 1.5868 - val_acc: 0.2143
Epoch 11/500
4/4 - 0s - loss: 1.4974 - acc: 0.3390 - val_loss: 1.5805 - val_acc: 0.1429
Epoch 12/500
4/4 - 0s - loss: 1.4055 - ac

**Best Accuracy Results for the data**
- loss: 0.4892 - acc: 0.8051 - val_loss: 1.4948 - val_acc: 0.7857

- loss: 0.4867 - acc: 0.8136 - val_loss: 1.4991 - val_acc: 0.7143